In [ ]:
# 인사아트센터 현재 전시 크롤링
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import time
from urllib.parse 

In [ ]:
import urljoin, urlparse
import re
from datetime import datetime

# 필요한 라이브러리 설치 (필요시)
# !pip install requests beautifulsoup4 pandas selenium pillow

print("라이브러리 import 완료")


In [ ]:
# 인사아트센터 홈페이지 접속 및 구조 분석
def get_page_content(url):
    """웹페이지 내용을 가져오는 함수"""
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        return response.text
    except requests.exceptions.RequestException as e:
        print(f"페이지 요청 오류: {e}")
        return None

# 인사아트센터 홈페이지 URL
base_url = "https://www.insaartcenter.com"
current_exhibitions_url = "https://www.insaartcenter.com/exhibition/current"

print(f"인사아트센터 홈페이지: {base_url}")
print(f"현재 전시 페이지: {current_exhibitions_url}")

# 페이지 내용 확인
page_content = get_page_content(current_exhibitions_url)
if page_content:
    print("페이지 로드 성공!")
    # HTML 구조 미리보기
    soup = BeautifulSoup(page_content, 'html.parser')
    print("\\n=== 페이지 제목 ===")
    print(soup.title.string if soup.title else "제목 없음")
else:
    print("페이지 로드 실패!")


In [ ]:
# HTML 구조 분석 및 전시 목록 추출
def analyze_page_structure(soup):
    """페이지 구조를 분석하여 전시 정보가 있는 부분을 찾는 함수"""
    print("\\n=== 페이지 구조 분석 ===")
    
    # 가능한 전시 관련 클래스나 태그들 찾기
    possible_selectors = [
        'div[class*="exhibition"]',
        'div[class*="current"]', 
        'div[class*="show"]',
        'article',
        '.exhibition-item',
        '.current-exhibition',
        '.exhibition-list'
    ]
    
    for selector in possible_selectors:
        elements = soup.select(selector)
        if elements:
            print(f"\\n발견된 요소: {selector} - {len(elements)}개")
            for i, elem in enumerate(elements[:3]):  # 처음 3개만 출력
                print(f"  [{i+1}] {elem.get_text(strip=True)[:100]}...")
    
    # 링크들 분석
    links = soup.find_all('a', href=True)
    exhibition_links = [link for link in links if any(keyword in link.get('href', '').lower() 
                       for keyword in ['exhibition', 'show', 'current', '전시'])]
    
    print(f"\\n전시 관련 링크 {len(exhibition_links)}개 발견:")
    for link in exhibition_links[:5]:  # 처음 5개만 출력
        href = link.get('href')
        text = link.get_text(strip=True)
        print(f"  - {text}: {href}")

if page_content:
    soup = BeautifulSoup(page_content, 'html.parser')
    analyze_page_structure(soup)


In [ ]:
# 전시 목록 크롤링 함수
def extract_exhibition_list(soup, base_url):
    """전시 목록을 추출하는 함수"""
    exhibitions = []
    
    # 다양한 선택자로 전시 정보 찾기
    selectors_to_try = [
        '.exhibition-item',
        '.current-exhibition',
        '.exhibition-card',
        '.exhibition-list .item',
        'article',
        '.show-item',
        'div[class*="exhibition"]'
    ]
    
    for selector in selectors_to_try:
        items = soup.select(selector)
        if items:
            print(f"\\n선택자 '{selector}'로 {len(items)}개 전시 발견")
            
            for item in items:
                exhibition_data = {}
                
                # 전시명 추출
                title_selectors = ['h1', 'h2', 'h3', '.title', '.exhibition-title', 'a']
                for title_sel in title_selectors:
                    title_elem = item.select_one(title_sel)
                    if title_elem and title_elem.get_text(strip=True):
                        exhibition_data['title'] = title_elem.get_text(strip=True)
                        break
                
                # 링크 추출
                link_elem = item.find('a', href=True)
                if link_elem:
                    href = link_elem.get('href')
                    if href:
                        exhibition_data['link'] = urljoin(base_url, href)
                
                # 이미지 추출
                img_elem = item.find('img')
                if img_elem:
                    img_src = img_elem.get('src') or img_elem.get('data-src')
                    if img_src:
                        exhibition_data['image_url'] = urljoin(base_url, img_src)
                
                # 전시 기간 추출
                period_patterns = [
                    r'\\d{4}\\.\\d{2}\\.\\d{2}.*?\\d{4}\\.\\d{2}\\.\\d{2}',
                    r'\\d{4}\\.\\d{1,2}\\.\\d{1,2}.*?\\d{4}\\.\\d{1,2}\\.\\d{1,2}',
                    r'\\d{4}-\\d{2}-\\d{2}.*?\\d{4}-\\d{2}-\\d{2}'
                ]
                
                item_text = item.get_text()
                for pattern in period_patterns:
                    match = re.search(pattern, item_text)
                    if match:
                        exhibition_data['period'] = match.group()
                        break
                
                # 작가명 추출 (보통 전시명 뒤에 나옴)
                artist_patterns = [
                    r'작가[:：]\\s*([^\\n]+)',
                    r'Artist[:：]\\s*([^\\n]+)',
                    r'Artist\\s*([^\\n]+)'
                ]
                
                for pattern in artist_patterns:
                    match = re.search(pattern, item_text, re.IGNORECASE)
                    if match:
                        exhibition_data['artist'] = match.group(1).strip()
                        break
                
                # 전시장소 추출
                venue_patterns = [
                    r'장소[:：]\\s*([^\\n]+)',
                    r'Venue[:：]\\s*([^\\n]+)',
                    r'Location[:：]\\s*([^\\n]+)'
                ]
                
                for pattern in venue_patterns:
                    match = re.search(pattern, item_text, re.IGNORECASE)
                    if match:
                        exhibition_data['venue'] = match.group(1).strip()
                        break
                
                # 유효한 데이터가 있는 경우만 추가
                if exhibition_data.get('title') or exhibition_data.get('link'):
                    exhibitions.append(exhibition_data)
            
            if exhibitions:
                break
    
    return exhibitions

# 전시 목록 추출 실행
if page_content:
    exhibitions = extract_exhibition_list(soup, base_url)
    print(f"\\n=== 추출된 전시 목록 ({len(exhibitions)}개) ===")
    for i, exhibition in enumerate(exhibitions, 1):
        print(f"\\n[{i}] 전시명: {exhibition.get('title', 'N/A')}")
        print(f"    링크: {exhibition.get('link', 'N/A')}")
        print(f"    기간: {exhibition.get('period', 'N/A')}")
        print(f"    작가: {exhibition.get('artist', 'N/A')}")
        print(f"    장소: {exhibition.get('venue', 'N/A')}")
        print(f"    이미지: {exhibition.get('image_url', 'N/A')}")
else:
    print("페이지 내용이 없어서 전시 목록을 추출할 수 없습니다.")


In [ ]:
# 개별 전시 상세 정보 크롤링 함수
def extract_exhibition_details(exhibition_url):
    """개별 전시의 상세 정보를 추출하는 함수"""
    if not exhibition_url:
        return {}
    
    print(f"\\n전시 상세 정보 크롤링: {exhibition_url}")
    
    page_content = get_page_content(exhibition_url)
    if not page_content:
        return {}
    
    soup = BeautifulSoup(page_content, 'html.parser')
    details = {}
    
    # 전시 설명 추출
    description_selectors = [
        '.exhibition-description',
        '.description',
        '.content',
        '.exhibition-content',
        '.show-description',
        'p',
        '.text-content'
    ]
    
    for selector in description_selectors:
        desc_elem = soup.select_one(selector)
        if desc_elem and desc_elem.get_text(strip=True):
            details['description'] = desc_elem.get_text(strip=True)
            break
    
    # 작품 이미지들 추출
    image_selectors = [
        '.exhibition-images img',
        '.gallery img',
        '.artwork-images img',
        '.show-images img',
        'img[class*="artwork"]',
        'img[class*="exhibition"]'
    ]
    
    artwork_images = []
    for selector in image_selectors:
        images = soup.select(selector)
        for img in images:
            img_src = img.get('src') or img.get('data-src')
            if img_src:
                full_url = urljoin(exhibition_url, img_src)
                artwork_images.append(full_url)
    
    if artwork_images:
        details['artwork_images'] = list(set(artwork_images))  # 중복 제거
    
    # 추가 정보 추출 (전시관, 관람시간 등)
    info_selectors = [
        '.exhibition-info',
        '.show-info',
        '.details',
        '.info'
    ]
    
    for selector in info_selectors:
        info_elem = soup.select_one(selector)
        if info_elem:
            info_text = info_elem.get_text(strip=True)
            details['additional_info'] = info_text
            break
    
    return details

# 전시 상세 정보 크롤링 실행
if page_content and exhibitions:
    print("\\n=== 전시 상세 정보 크롤링 시작 ===")
    
    for i, exhibition in enumerate(exhibitions):
        if exhibition.get('link'):
            details = extract_exhibition_details(exhibition['link'])
            
            # 기본 정보에 상세 정보 추가
            if details:
                exhibition.update(details)
            
            print(f"\\n[{i+1}] 전시 상세 정보 업데이트 완료")
            if details.get('description'):
                print(f"    설명: {details['description'][:100]}...")
            if details.get('artwork_images'):
                print(f"    작품 이미지: {len(details['artwork_images'])}개")
            
            # 서버 부하 방지를 위한 대기
            time.sleep(1)
    
    print("\\n전시 상세 정보 크롤링 완료!")
else:
    print("전시 목록이 없어서 상세 정보를 크롤링할 수 없습니다.")


In [ ]:
# 작품 사진 다운로드 함수
def download_image(image_url, save_path, headers=None):
    """이미지를 다운로드하는 함수"""
    if not image_url:
        return False
    
    try:
        if headers is None:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
            }
        
        response = requests.get(image_url, headers=headers, timeout=30)
        response.raise_for_status()
        
        # 파일 확장자 확인
        parsed_url = urlparse(image_url)
        file_extension = os.path.splitext(parsed_url.path)[1]
        if not file_extension:
            file_extension = '.jpg'  # 기본값
        
        # 파일명 생성 (URL에서 안전한 파일명 추출)
        filename = os.path.basename(parsed_url.path)
        if not filename or filename == '/':
            filename = f"image_{hash(image_url) % 10000}{file_extension}"
        
        full_path = os.path.join(save_path, filename)
        
        with open(full_path, 'wb') as f:
            f.write(response.content)
        
        return full_path
        
    except Exception as e:
        print(f"이미지 다운로드 실패 ({image_url}): {e}")
        return False

def download_exhibition_images(exhibitions, base_save_dir="exhibition_images"):
    """전시 관련 이미지들을 다운로드하는 함수"""
    if not os.path.exists(base_save_dir):
        os.makedirs(base_save_dir)
    
    downloaded_images = {}
    
    for i, exhibition in enumerate(exhibitions):
        exhibition_title = exhibition.get('title', f'exhibition_{i+1}')
        # 파일명에 사용할 수 없는 문자 제거
        safe_title = re.sub(r'[<>:"/\\\\|?*]', '_', exhibition_title)
        exhibition_dir = os.path.join(base_save_dir, safe_title)
        
        if not os.path.exists(exhibition_dir):
            os.makedirs(exhibition_dir)
        
        exhibition_images = []
        
        # 메인 이미지 다운로드
        if exhibition.get('image_url'):
            main_image_path = download_image(exhibition['image_url'], exhibition_dir)
            if main_image_path:
                exhibition_images.append(main_image_path)
                print(f"메인 이미지 다운로드 완료: {main_image_path}")
        
        # 작품 이미지들 다운로드
        if exhibition.get('artwork_images'):
            for j, artwork_url in enumerate(exhibition['artwork_images']):
                artwork_path = download_image(artwork_url, exhibition_dir)
                if artwork_path:
                    exhibition_images.append(artwork_path)
                    print(f"작품 이미지 {j+1} 다운로드 완료: {artwork_path}")
                
                # 서버 부하 방지
                time.sleep(0.5)
        
        downloaded_images[exhibition_title] = exhibition_images
    
    return downloaded_images

# 이미지 다운로드 실행 (옵션)
print("\\n=== 작품 이미지 다운로드 ===")
print("이미지 다운로드를 시작하려면 아래 셀을 실행하세요.")
print("주의: 많은 이미지를 다운로드할 수 있으므로 시간이 오래 걸릴 수 있습니다.")


In [ ]:
# 이미지 다운로드 실행 (실제 다운로드를 원할 경우 아래 주석 해제)
# downloaded_images = download_exhibition_images(exhibitions)
# print(f"\\n총 {sum(len(images) for images in downloaded_images.values())}개의 이미지 다운로드 완료!")

# 현재는 다운로드하지 않고 URL만 표시
print("\\n=== 이미지 URL 목록 ===")
for i, exhibition in enumerate(exhibitions, 1):
    print(f"\\n[{i}] {exhibition.get('title', 'N/A')}")
    if exhibition.get('image_url'):
        print(f"    메인 이미지: {exhibition['image_url']}")
    if exhibition.get('artwork_images'):
        for j, img_url in enumerate(exhibition['artwork_images'], 1):
            print(f"    작품 이미지 {j}: {img_url}")
    else:
        print("    작품 이미지: 없음")


In [ ]:
# 데이터 정리 및 저장
def save_to_csv(exhibitions, filename="insa_art_center_exhibitions.csv"):
    """전시 데이터를 CSV 파일로 저장"""
    if not exhibitions:
        print("저장할 전시 데이터가 없습니다.")
        return
    
    # 데이터 정리
    cleaned_data = []
    for exhibition in exhibitions:
        cleaned_item = {
            '전시명': exhibition.get('title', ''),
            '전시기간': exhibition.get('period', ''),
            '작가': exhibition.get('artist', ''),
            '전시장소': exhibition.get('venue', ''),
            '전시관': exhibition.get('gallery', ''),
            '전시설명': exhibition.get('description', ''),
            '추가정보': exhibition.get('additional_info', ''),
            '전시링크': exhibition.get('link', ''),
            '메인이미지': exhibition.get('image_url', ''),
            '작품이미지수': len(exhibition.get('artwork_images', [])),
            '작품이미지URLs': '; '.join(exhibition.get('artwork_images', [])),
            '크롤링일시': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }
        cleaned_data.append(cleaned_item)
    
    # DataFrame 생성 및 저장
    df = pd.DataFrame(cleaned_data)
    df.to_csv(filename, index=False, encoding='utf-8-sig')
    
    print(f"\\n데이터가 '{filename}' 파일로 저장되었습니다.")
    print(f"총 {len(cleaned_data)}개의 전시 정보가 저장되었습니다.")
    
    return df

def save_to_json(exhibitions, filename="insa_art_center_exhibitions.json"):
    """전시 데이터를 JSON 파일로 저장"""
    if not exhibitions:
        print("저장할 전시 데이터가 없습니다.")
        return
    
    # 크롤링 시간 추가
    data_with_metadata = {
        'crawl_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'source': '인사아트센터',
        'total_exhibitions': len(exhibitions),
        'exhibitions': exhibitions
    }
    
    import json
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(data_with_metadata, f, ensure_ascii=False, indent=2)
    
    print(f"\\n데이터가 '{filename}' 파일로 저장되었습니다.")

# 데이터 저장 실행
if exhibitions:
    print("\\n=== 데이터 저장 ===")
    
    # CSV 저장
    df = save_to_csv(exhibitions)
    
    # JSON 저장
    save_to_json(exhibitions)
    
    # 요약 정보 출력
    print("\\n=== 크롤링 결과 요약 ===")
    print(f"총 전시 개수: {len(exhibitions)}")
    
    for i, exhibition in enumerate(exhibitions, 1):
        print(f"\\n[{i}] {exhibition.get('title', 'N/A')}")
        print(f"    기간: {exhibition.get('period', 'N/A')}")
        print(f"    작가: {exhibition.get('artist', 'N/A')}")
        print(f"    장소: {exhibition.get('venue', 'N/A')}")
        print(f"    설명: {exhibition.get('description', 'N/A')[:100]}...")
        print(f"    작품이미지: {len(exhibition.get('artwork_images', []))}개")
    
    # DataFrame 미리보기
    if df is not None:
        print("\\n=== DataFrame 미리보기 ===")
        print(df.head())
        
else:
    print("저장할 전시 데이터가 없습니다.")


In [ ]:
# 전체 크롤링 프로세스 실행 함수
def run_full_crawling():
    """전체 크롤링 프로세스를 실행하는 함수"""
    print("=== 인사아트센터 전시 크롤링 시작 ===")
    
    # 1. 페이지 로드
    page_content = get_page_content(current_exhibitions_url)
    if not page_content:
        print("페이지 로드 실패!")
        return None
    
    # 2. 전시 목록 추출
    soup = BeautifulSoup(page_content, 'html.parser')
    exhibitions = extract_exhibition_list(soup, base_url)
    
    if not exhibitions:
        print("전시 목록을 찾을 수 없습니다.")
        return None
    
    # 3. 상세 정보 크롤링
    print("\\n전시 상세 정보 크롤링 중...")
    for i, exhibition in enumerate(exhibitions):
        if exhibition.get('link'):
            details = extract_exhibition_details(exhibition['link'])
            if details:
                exhibition.update(details)
            time.sleep(1)  # 서버 부하 방지
    
    # 4. 데이터 저장
    print("\\n데이터 저장 중...")
    df = save_to_csv(exhibitions)
    save_to_json(exhibitions)
    
    print("\\n=== 크롤링 완료 ===")
    return exhibitions

# 실행 예시 (필요시 주석 해제)
# exhibitions = run_full_crawling()

print("\\n크롤링 코드가 준비되었습니다!")
print("실행하려면 위의 'run_full_crawling()' 함수를 호출하세요.")
print("\\n주의사항:")
print("1. 웹사이트 구조가 변경될 수 있으므로 선택자를 조정해야 할 수 있습니다.")
print("2. 이미지 다운로드는 시간이 오래 걸릴 수 있습니다.")
print("3. 웹사이트의 robots.txt와 이용약관을 확인하세요.")
print("4. 과도한 요청으로 서버에 부하를 주지 않도록 주의하세요.")
